In [3]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap

DATA_FILE_PATH = "./clean_data.csv"

In [5]:
conn = sqlite3.connect("usa_pollution.db")
df = pd.read_csv(DATA_FILE_PATH)

In [7]:
cursor = conn.cursor()

# Create locations table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS locations (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        state TEXT NOT NULL,
        county TEXT NOT NULL,
        city TEXT NOT NULL,
        latitude REAL NOT NULL,
        longitude REAL NOT NULL
    )
''')

In [9]:
# Create pollutants table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS pollutants (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        location_id INTEGER NOT NULL,
        year INTEGER NOT NULL,
        month INTEGER NOT NULL,
        day INTEGER NOT NULL,
        o3_mean REAL,
        o3_max_value REAL,
        o3_first_max_hour REAL,
        o3_aqi REAL,
        co_mean REAL,
        co_max_value REAL,
        co_first_max_hour REAL,
        co_aqi REAL,
        so2_mean REAL,
        so2_max_value REAL,
        so2_first_max_hour REAL,
        so2_aqi REAL,
        no2_mean REAL,
        no2_max_value REAL,
        no2_first_max_hour REAL,
        no2_aqi REAL,
        FOREIGN KEY(location_id) REFERENCES locations(id)
    )
''')

In [11]:
conn.commit()

In [13]:
cursor = conn.cursor()

location_columns = ["State", "County", "City", "Latitude", "Longitude"]
locations_df = df[location_columns].drop_duplicates().reset_index(drop=True)

locations_df.to_sql("locations", conn, if_exists="append", index=False)

location_query = "SELECT id, state, county, city, latitude, longitude FROM locations"
location_df = pd.read_sql(location_query, conn)

location_df.columns = ['id',"State", "County", "City", "Latitude", "Longitude"]

df = df.merge(location_df, on=["State", "County", "City", "Latitude", "Longitude"], how="left")
df.rename(columns={"id": "location_id"}, inplace=True)

pollutant_columns = [
    "O3 Mean", "O3 1st Max Value", "O3 1st Max Hour", "O3 AQI",
    "CO Mean", "CO 1st Max Value", "CO 1st Max Hour", "CO AQI",
    "SO2 Mean", "SO2 1st Max Value", "SO2 1st Max Hour", "SO2 AQI",
    "NO2 Mean", "NO2 1st Max Value", "NO2 1st Max Hour", "NO2 AQI",
    "Year", "Month", "Day", "location_id"
]

pollutants_df = df[pollutant_columns]

pollutants_df.columns = [
    "o3_mean", "o3_max_value", "o3_first_max_hour", "o3_aqi",
    "co_mean", "co_max_value", "co_first_max_hour", "co_aqi",
    "so2_mean", "so2_max_value", "so2_first_max_hour", "so2_aqi",
    "no2_mean", "no2_max_value", "no2_first_max_hour", "no2_aqi",
    "year", "month", "day", "location_id"
]

pollutants_df.to_sql("pollutants", conn, if_exists="append", index=False)

In [ ]:
# Example SQL Query: Average O3 AQI per State
query = """
SELECT l.state, AVG(p.o3_aqi) as avg_o3_aqi
FROM pollutants p
JOIN locations l ON p.location_id = l.id
GROUP BY l.state
ORDER BY avg_o3_aqi DESC
LIMIT 10;
"""

In [ ]:
# Execute the query and load the results into a Pandas DataFrame
df_sql = pd.read_sql_query(query, conn)

#Visualize the results
plt.figure(figsize=(12, 6))
plt.barh(df_sql["state"], df_sql["avg_o3_aqi"], color="skyblue")
plt.xlabel("Average O3 AQI")
plt.ylabel("State")
plt.title("Top 10 States with Highest Average O3 AQI")
plt.gca().invert_yaxis()  # Invert y-axis for better readability
plt.show()

In [ ]:
# SQL Query to fetch pollution data with latitude & longitude
query = """
SELECT l.latitude, l.longitude, p.o3_aqi
FROM pollutants p
JOIN locations l ON p.location_id = l.id
WHERE p.o3_aqi IS NOT NULL;
"""

In [ ]:
# Execute the query and load data into Pandas DataFrame
df_heatmap = pd.read_sql_query(query, conn)

In [ ]:
# Create a base map centered in the USA
m = folium.Map(location=[37.0902, -95.7129], zoom_start=5)

# Add HeatMap layer
heat_data = df_heatmap[["latitude", "longitude", "o3_aqi"]].values.tolist()
HeatMap(heat_data, radius=15, blur=25, max_zoom=1).add_to(m)

# Display the map
m

In [ ]:
conn.close()